In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind 

In [143]:
df = pd.read_csv('listings.csv', low_memory=False)

In [144]:
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,20191204162709,2019-12-04,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,...,f,f,moderate,f,f,1,1,0,0,1.79
1,3850,https://www.airbnb.com/rooms/3850,20191204162709,2019-12-04,Charming room for two,Your own private room plus access to a shared ...,This room can fit two people. Nobody else will...,Your own private room plus access to a shared ...,none,"This is a quiet, safe neighborhood on a substa...",...,f,f,strict_14_with_grace_period,f,f,3,0,3,0,1.38
2,5858,https://www.airbnb.com/rooms/5858,20191204162709,2019-12-05,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,none,I love how our neighborhood feels quiet but is...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.86
3,7918,https://www.airbnb.com/rooms/7918,20191204162709,2019-12-04,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,"Settle down, S.F. resident, student, hospital,...",Nice and good public transportation. 7 minute...,none,"Shopping old town, restaurants, McDonald, Whol...",...,f,f,strict_14_with_grace_period,f,f,9,0,9,0,0.14
4,8142,https://www.airbnb.com/rooms/8142,20191204162709,2019-12-04,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,"Settle down, S.F. resident, student, hospital,...",Nice and good public transportation. 7 minute...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,9,0,9,0,0.13


In [145]:
df.shape #get the number of columns and rows in dataset

(8533, 106)

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8533 entries, 0 to 8532
Columns: 106 entries, id to reviews_per_month
dtypes: float64(23), int64(21), object(62)
memory usage: 6.9+ MB


Step 1: Drop obvious non-useful columns

In [147]:
df = df.drop(['scrape_id','last_scraped','thumbnail_url','medium_url','picture_url','xl_picture_url','host_name',
         'host_thumbnail_url','neighbourhood_group_cleansed'], axis=1)

In [148]:
df.shape #output: dropped 9 columns 

(8533, 97)

Step 2: Investigating NULL columns

In [149]:
df.dropna(how = 'any').shape #would result in these many rows if we drop all rows that has any null columns 

(0, 97)

In [150]:
df.columns[df.isna().any()].tolist() #selecting the columns with null values

['summary',
 'space',
 'description',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'city',
 'state',
 'zipcode',
 'market',
 'bathrooms',
 'bedrooms',
 'beds',
 'square_feet',
 'weekly_price',
 'monthly_price',
 'security_deposit',
 'cleaning_fee',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'license',
 'jurisdiction_names',
 'reviews_per_month']

In [151]:
len(df.columns[df.isna().any()].tolist())#counting how many columns have NULL values

47

In [152]:
df[df.columns[df.isna().any()].tolist()].isna().sum().sort_values(ascending = False)

host_acceptance_rate           8533
square_feet                    8407
monthly_price                  7507
weekly_price                   7480
notes                          3317
access                         3244
license                        3243
interaction                    2804
transit                        2606
house_rules                    2283
host_about                     2155
neighborhood_overview          2111
review_scores_location         1938
review_scores_value            1938
review_scores_checkin          1938
review_scores_accuracy         1937
review_scores_cleanliness      1937
review_scores_communication    1936
review_scores_rating           1932
first_review                   1891
last_review                    1891
reviews_per_month              1891
security_deposit               1759
space                          1208
host_response_time             1000
host_response_rate             1000
cleaning_fee                    959
host_neighbourhood          

In [153]:
cols_num_missing = df[df.columns[df.isna().any()].tolist()].isna().sum().sort_values(ascending = False)


In [154]:
cols_num_missing[cols_num_missing <= 60]

host_location                59
host_picture_url             54
host_listings_count          54
host_total_listings_count    54
host_has_profile_pic         54
host_identity_verified       54
host_since                   54
host_is_superhost            54
jurisdiction_names           51
description                  41
market                       20
beds                         12
neighbourhood                10
city                         10
state                         7
bathrooms                     5
bedrooms                      3
dtype: int64

In [155]:
less_than_60_null = [
'host_location',
'host_picture_url',
'host_listings_count',
'host_total_listings_count',
'host_has_profile_pic',
'host_identity_verified',
'host_since',
'host_is_superhost',
'jurisdiction_names',
'description',
'market',
'beds',
'neighbourhood',
'city',
'state',
'bathrooms',
'bedrooms',
]

In [156]:
cols_num_missing

host_acceptance_rate           8533
square_feet                    8407
monthly_price                  7507
weekly_price                   7480
notes                          3317
access                         3244
license                        3243
interaction                    2804
transit                        2606
house_rules                    2283
host_about                     2155
neighborhood_overview          2111
review_scores_location         1938
review_scores_value            1938
review_scores_checkin          1938
review_scores_accuracy         1937
review_scores_cleanliness      1937
review_scores_communication    1936
review_scores_rating           1932
first_review                   1891
last_review                    1891
reviews_per_month              1891
security_deposit               1759
space                          1208
host_response_time             1000
host_response_rate             1000
cleaning_fee                    959
host_neighbourhood          

In [157]:
#2.1) Drop the columns with many NULL values, there are 8533 rows in total 

### first drop the obvious one
df2 = df.drop(['host_acceptance_rate'], axis = 1)

In [158]:
### second drop the columns with majority null
null_perc = cols_num_missing/len(df2) 

In [159]:
null_perc

host_acceptance_rate           1.000000
square_feet                    0.985234
monthly_price                  0.879761
weekly_price                   0.876597
notes                          0.388726
access                         0.380171
license                        0.380054
interaction                    0.328607
transit                        0.305403
house_rules                    0.267550
host_about                     0.252549
neighborhood_overview          0.247392
review_scores_location         0.227118
review_scores_value            0.227118
review_scores_checkin          0.227118
review_scores_accuracy         0.227001
review_scores_cleanliness      0.227001
review_scores_communication    0.226884
review_scores_rating           0.226415
first_review                   0.221610
last_review                    0.221610
reviews_per_month              0.221610
security_deposit               0.206141
space                          0.141568
host_response_time             0.117192


In [160]:
df2 = df.drop(['host_acceptance_rate','square_feet','monthly_price','weekly_price'], axis = 1)

In [161]:
df2.shape 

(8533, 93)

In [162]:
#2.2) Drop rows for columns that are missing <= 60 rows
# df2['host_location'].head(10)
# less_than_60_null.drop
df2.dropna(subset = less_than_60_null, inplace= True)

In [186]:
df2.shape 

(8334, 93)

Step 3: Check validity of certain data points

In [163]:
#3.1: check validity of NULL zip codes 
df2[df2['zipcode'].isnull()]['neighbourhood'].value_counts()

SoMa                     49
Nob Hill                 34
Western Addition/NOPA    31
Downtown                 19
Mission District         10
Potrero Hill              9
Russian Hill              8
Financial District        8
Dogpatch                  7
The Castro                6
Pacific Heights           6
Cole Valley               5
Bernal Heights            4
Hayes Valley              4
Richmond District         4
Noe Valley                3
Duboce Triangle           3
South Beach               3
Lakeshore                 3
Alamo Square              2
Civic Center              2
North Beach               2
Telegraph Hill            2
Outer Sunset              1
Bayview                   1
Presidio Heights          1
Marina                    1
Crocker Amazon            1
Fisherman's Wharf         1
Twin Peaks                1
Haight-Ashbury            1
Chinatown                 1
Lower Haight              1
Parkside                  1
Name: neighbourhood, dtype: int64

In [164]:
df2['neighbourhood'].value_counts()

Mission District         777
SoMa                     635
Western Addition/NOPA    463
Richmond District        433
Outer Sunset             403
Bernal Heights           394
Downtown                 391
Nob Hill                 381
Noe Valley               354
The Castro               344
Haight-Ashbury           231
Bayview                  219
Potrero Hill             209
Pacific Heights          175
Tenderloin               175
Union Square             157
Inner Sunset             156
Telegraph Hill           147
Duboce Triangle          146
Excelsior                133
Cole Valley              117
Sunnyside                117
South Beach              116
Russian Hill             110
Hayes Valley             104
Marina                   101
Lower Haight              87
Chinatown                 76
Financial District        76
Crocker Amazon            74
Portola                   72
Glen Park                 70
Mission Terrace           69
Twin Peaks                69
Cow Hollow    

In [165]:
pd.pivot_table(df2[['zipcode','neighbourhood','availability_365']],index=['neighbourhood','zipcode'])
#each neighborhood as multiple zip codes so didn't assign the null zipcodes zipcodes according to their neighborhood

availability_365
neighbourhood         zipcode                   
Alamo Square          94115           121.333333
                      94117           196.392157
Balboa Terrace        94112           199.115385
                      94127           266.444444
Bayview               94110           365.000000
...                                          ...
Western Addition/NOPA 94121           327.000000
                      CA 94102        237.250000
                      CA 94109        240.250000
                      CA 94115        239.000000
                      CA 94117        321.000000

[165 rows x 1 columns]

Step 4: Reformatting Columns

In [166]:
#formatting the numeric columns 
numeric_cols = ['price','security_deposit','cleaning_fee','extra_people']

In [167]:
#reformat zip_code

#strip out the $ in the numeric columns

for col in numeric_cols: 
    df2[col] = df2[col].str.lstrip('$')
    df2[col] = df2[col].str.replace(',','').astype(float)
#     df2[col] = df2[col].strip(',')

In [168]:
df2[numeric_cols].head(10)

,price,security_deposit,cleaning_fee,extra_people
0,170.0,100.0,100.0,25.0
1,99.0,0.0,10.0,20.0
2,235.0,NaN,100.0,0.0
3,65.0,200.0,50.0,12.0
4,65.0,200.0,50.0,12.0
5,585.0,0.0,175.0,300.0
6,139.0,0.0,50.0,60.0
7,135.0,NaN,50.0,0.0
8,199.0,500.0,100.0,0.0
9,120.0,500.0,75.0,0.0


In [169]:
#changing object to integers
df2[numeric_cols].apply(pd.to_numeric, errors = 'coerce') #making it into a numeric 

,price,security_deposit,cleaning_fee,extra_people
0,170.0,100.0,100.0,25.0
1,99.0,0.0,10.0,20.0
2,235.0,NaN,100.0,0.0
3,65.0,200.0,50.0,12.0
4,65.0,200.0,50.0,12.0
...,...,...,...,...
8517,60.0,0.0,0.0,0.0
8518,60.0,0.0,0.0,0.0
8519,385.0,800.0,185.0,25.0
8520,175.0,NaN,NaN,0.0


In [170]:
df2[numeric_cols].dtypes

price               float64
security_deposit    float64
cleaning_fee        float64
extra_people        float64
dtype: object

Step 5: Checking the Security Deposit & Cleaning Fee

In [172]:
## mean of price with $0 security deposit
df2[['price']][df2['security_deposit'] == 0].mean()

price    177.974008
dtype: float64

In [175]:
## mean of price with > $0 security deposit
df2[['price']][df2['security_deposit'] != 0].mean()

price    230.304348
dtype: float64

In [176]:
## mean of price with NULL security despoit 
df2[['price']][df2['security_deposit'].isnull()].mean()

price    219.71017
dtype: float64

In [177]:
## mean of price with $0 cleaning fee
df2[['price']][df2['cleaning_fee'] == 0].mean()

price    224.296524
dtype: float64

In [178]:
## mean of price with > $0 security deposit
df2[['price']][df2['cleaning_fee'] != 0].mean()

price    216.050351
dtype: float64

In [179]:
## mean of price with NULL security despoit 
df2[['price']][df2['cleaning_fee'].isnull()].mean()

price    241.489617
dtype: float64

Security Deposit

In [180]:
## testing $0 & NULL 
ttest_ind(df2[['price']][df2['security_deposit'] == 0], df2[['price']][df2['security_deposit'].isnull()])

Ttest_indResult(statistic=array([-4.58754434]), pvalue=array([4.62610398e-06]))

In [181]:
# testing $0 & NonZero
ttest_ind(df2[['price']][df2['security_deposit'] == 0],df2[['price']][df2['security_deposit'] != 0])

Ttest_indResult(statistic=array([-6.96365472]), pvalue=array([3.56722749e-12]))

In [182]:
# testing NonZero & NULL
ttest_ind(df2[['price']][df2['security_deposit'].isnull()],df2[['price']][df2['security_deposit'] != 0])

Ttest_indResult(statistic=array([-1.19165111]), pvalue=array([0.23343415]))

Cleaning Fee

In [183]:
## testing $0 & NULL 
ttest_ind(df2[['price']][df2['cleaning_fee'] == 0], df2[['price']][df2['cleaning_fee'].isnull()])

Ttest_indResult(statistic=array([-0.73123489]), pvalue=array([0.46475788]))

In [185]:
# testing $0 & NonZero
ttest_ind(df2[['price']][df2['cleaning_fee'] == 0],df2[['price']][df2['cleaning_fee'] != 0])

Ttest_indResult(statistic=array([0.58397856]), pvalue=array([0.55925055]))

In [184]:
# testing NonZero & NULL
ttest_ind(df2[['price']][df2['cleaning_fee'].isnull()],df2[['price']][df2['cleaning_fee'] != 0])

Ttest_indResult(statistic=array([2.27006461]), pvalue=array([0.02322786]))

Read to CSV File

In [187]:
df2.to_csv('cleaned_listing.csv')